In [1]:
from path_reference.folder_reference import get_data_path
from webscrapping.witcher_scrapping import WitcherScrapper
from data.df_cleanup import cleanup_df
from pathlib import Path
from selenium.webdriver.common.by import By
import os
import spacy
from spacy import displacy
from nlp.relationship_creator import RelationshipCreator
import pandas as pd

In [2]:
rc = RelationshipCreator()
window_df = rc.aggregate_network()

In [3]:
window_df

,source,target,occurrences
0,Geralt,Velerad,52
1,Ostrit,Velerad,61
2,Adda,Ostrit,35
3,Adda,Segelin,3
4,Ostrit,Segelin,4
...,...,...,...
264,Count,Geralt,903
265,Dennis,Geralt,6413
266,Dennis,Falwick,1822
267,Count,Dennis,3683


In [4]:
import numpy as np
np.sort(window_df.values, axis=1)

array([['Geralt', 'Velerad'],
       ['Ostrit', 'Velerad'],
       ['Adda', 'Ostrit'],
       ...,
       ['Geralt', 'Nenneke'],
       ['Iola', 'Nenneke'],
       ['Iola', 'Nenneke']], dtype=object)

In [31]:
windows_size = 5
maximum_df_index = len(entity_df)

for i in range(entity_df.index[-1]):
    window_end = min(i+5, maximum_df_index)
    window = entity_df.loc[i:i+5]
    window_characters = sum(window.character_entities, [])
    maximum_size = len(window_characters)-1
    unique_characters = []

    for index, item in enumerate(window_characters):
        current_item = item
        next_item = window_characters[min(index+1, maximum_size)]
        if current_item != next_item:
            unique_characters.extend((current_item, next_item))
    if 200 < i < 300:
        print(unique_characters)

[]
[]
[]
[]
[]
[]
[]
['Geralt', 'Fenne']
['Geralt', 'Fenne']
['Fenne', 'Geralt']
['Fenne', 'Geralt', 'Geralt', 'Fenne']
['Fenne', 'Geralt', 'Geralt', 'Fenne']
['Fenne', 'Geralt', 'Geralt', 'Fenne']
['Geralt', 'Fenne', 'Fenne', 'Nivellen']
['Geralt', 'Fenne', 'Fenne', 'Nivellen', 'Nivellen', 'Primula']
['Fenne', 'Nivellen', 'Nivellen', 'Primula', 'Primula', 'Geralt']
['Fenne', 'Nivellen', 'Nivellen', 'Primula', 'Primula', 'Geralt', 'Geralt', 'Primula']
['Fenne', 'Nivellen', 'Nivellen', 'Primula', 'Primula', 'Geralt', 'Geralt', 'Primula', 'Primula', 'Geralt']
['Nivellen', 'Primula', 'Primula', 'Geralt', 'Geralt', 'Primula', 'Primula', 'Geralt']
['Primula', 'Geralt', 'Geralt', 'Primula', 'Primula', 'Geralt', 'Geralt', 'Nivellen']
['Geralt', 'Primula', 'Primula', 'Geralt', 'Geralt', 'Nivellen', 'Nivellen', 'Geralt']
['Primula', 'Geralt', 'Geralt', 'Nivellen', 'Nivellen', 'Geralt', 'Geralt', 'two', 'two', 'Primula', 'Primula', 'Ilka', 'Ilka', 'Venimira']
['Geralt', 'Nivellen', 'Nivellen', '

In [2]:
characters = pd.read_csv(Path(get_data_path(), "characters.csv"))
#cleanup_df(characters)

In [12]:
#first_names = [x.lower() for x in characters["character_first_name"].tolist()]
characters[characters["character_first_name"].str.lower() == "two"]

,book,character,character_first_name
1255,Time of Contempt,Two Tusks,Two


In [11]:
sp1 = "['Geralt']"
forbidden_characters = ["[", "'", "]"]
for c in forbidden_characters:
    sp1 = sp1.replace(c, "")
sp1

'Geralt'

In [9]:
sp1 = "geralt, yennefer"
sp1.split(", ")

['geralt', 'yennefer']

In [23]:
def filter_entity_df(entity_list: list[str], characters_df: pd.DataFrame):
    first_names = [x.lower() for x in characters_df["character_first_name"].tolist()]
    full_names = [x.lower() for x in characters_df["character"].tolist()]
    return [entity for entity in entity_list
            if entity.lower() in first_names
            or entity.lower() in full_names]

sent_entity_df["character_entities"] = sent_entity_df["entities"].apply(
            lambda x: filter_entity_df(x, characters))

In [25]:
# Filter sent_entity_df where character_entities values are null
sent_entity_df = sent_entity_df[sent_entity_df["character_entities"].map(len) > 0]
sent_entity_df

,sentence,entities,character_entities
0,"(\n\n, ', Come, on, ,, Geralt, ,, it, 'll, be,...",[Geralt],[Geralt]
1,"(Damned, mother's-, !, ', \n\n, ', Careful, ,,...","[Velerad, Ostrit]","[Velerad, Ostrit]"
2,"(', Say, what, you, want, about, the, striga, ...",[Adda],[Adda]
3,"(Segelin, and, Ostrit, looked, at, each, other...","[Segelin, Ostrit]","[Segelin, Ostrit]"
4,"(', At, the, very, beginning, ,, seven, years,...","[seven years ago, two]",[two]
...,...,...,...
1868,"(Nenneke, raised, herself, to, her, knees, and...",[Nenneke],[Nenneke]
1870,"(She, turned, to, Geralt, .)",[Geralt],[Geralt]
1871,"(Iola-, ', \n\n, ', Do, n't, say, anything, ,,...",[Nenneke],[Nenneke]
1873,"(See, to, Iola, .)",[Iola],[Iola]


In [36]:
x = ["Geralt of Rivia"]
[item.split()[0] for item in x]

['Geralt']

In [15]:
characters

,book,character,character_first_name
0,Baptism of Fire,Adalia,Adalia
1,Baptism of Fire,Adela,Adela
2,Baptism of Fire,Aen Saevherne,Aen
3,Baptism of Fire,Aevenien,Aevenien
4,Baptism of Fire,Aglaïs,Aglaïs
...,...,...,...
1270,Time of Contempt,Yanna of Murivel,Yanna
1271,Time of Contempt,Yarpen Zigrin,Yarpen
1272,Time of Contempt,Yennefer of Vengerberg,Yennefer
1273,Time of Contempt,Yiolenta Suarez,Yiolenta
